# PMAC: Historical Comparisons

In [1]:
import pandas as pd

import pmac_utils
from shared_utils import rt_dates

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
dfs = {}
for key, date in rt_dates.PMAC.items():
    df = pmac_utils.flag_parallel_intersecting_routes(date)
    summary = pmac_utils.get_summary_table(df) # df[df._merge=="both"]?
    dfs[key] = summary

In [3]:
dfs.keys()

dict_keys(['Q1_2022', 'Q2_2022'])

In [4]:
dfs["Q1_2022"].total_service_hours.sum()

106445

In [5]:
dfs["Q2_2022"].total_service_hours.sum()

109560

In [6]:
def change_from_prior(current: pd.DataFrame, prior: pd.DataFrame, 
                      col: str) -> pd.DataFrame:
    keep_cols = ["category", col]
    
    current2 = current[keep_cols].rename(columns={col: "current"})
    prior2 = prior[keep_cols].rename(columns={col: "prior"})
                  
    df = pd.merge(current2, prior2, on = "category", how = "inner")
    
    df = df.assign(
        change = df.current - df.prior,
        pct_change = (df.current - df.prior).divide(df.prior)
    )
    
    return df

In [7]:
change_from_prior(dfs["Q2_2022"], dfs["Q1_2022"], "total_service_hours")

,category,current,prior,change,pct_change
0,parallel,59587,45860,13727,0.299324
1,other,30414,40757,-10343,-0.253772
2,on_shn,19559,19828,-269,-0.013567
